# Simulating wind and marine hydrokinetic turbines with actuator lines in RANS and LES

<p class="gap3"<p>

<center>
<img width=200px src="figures/unh.png">
</center>

<p class="gap3"<p>

<center>
Pete Bachant (<a href="mailto:petebachant@gmail.com">petebachant@gmail.com</a>)
<p>
</center>

<center>
Martin Wosnik (<a href="martin.wosnik@unh.edu">martin.wosnik@unh.edu</a>)
<p>
</center>

In [ ]:
# Setup stuff
%load_ext autoreload
%autoreload 2
import io
import base64
from IPython.display import HTML
from importlib.machinery import SourceFileLoader
%matplotlib inline
import os
talk_dir = os.getcwd()
import matplotlib.pyplot as plt
import seaborn as sns
from pxl.styleplot import set_sns

# Set plot styling
set_sns(context="talk", rc={"lines.markersize": 14, "lines.markeredgewidth": 2, "axes.grid": True, 
                            "font.size": 1.5*14.3})

# Define some directories
exp_dir = "C:/Users/Pete/Research/Experiments"
rvat_baseline_dir = "C:/Users/Pete/Research/Experiments/RVAT baseline"
rvat_re_dep_dir = os.path.join(exp_dir, "RVAT Re dep")

def embed_video(fpath):
    video = io.open(fpath, 'r+b').read()
    encoded = base64.b64encode(video)
    return HTML(data='''<center><video controls loop>
                        <source src="data:video/mp4;base64,{0}" type="video/mp4" />
                     </video></center>'''.format(encoded.decode('ascii')))


## Motivation

* Experimental results showing promise for CFTs in arrays ([Dabiri (2011)](http://dx.doi.org/10.1063/1.3608170), [Kinzel et al. (2012)](http://dx.doi.org/10.1080/14685248.2012.712698))
* Vertical mean velocity field largest near-wake recovery mechanism for high-solidity CFTs ([Bachant and Wosnik (2015)](http://dx.doi.org/10.1080/14685248.2014.1001852))
* 2-D blade-resolved RANS can't resolve vertical velocity
* 3-D blade-resolved too expensive
* ALM for CFT turbine and array analysis/design?

## ALM survey

| Author(s) | Code(s) | CFT? | Open? |
|-----------|------|:------:|:-----------------:|
| Sorensen and Shen (2002) | EllipSys3D | ![](figures/x.png) | ![](figures/x.png) |
| Keck (2012) | ANSYS-CFX + ? | ![](figures/x.png) | ![](figures/x.png) |
| Schluntz and Willden (2014) | FLUENT + ? | ![](figures/x.png) | ![](figures/x.png) |
| Shamsoddin and Porté-Agel (2014) | In-house | ![](figures/check.png) | ![](figures/x.png) |
| Zhong et al. (2015) | OpenFOAM + ? | ![](figures/x.png) | ![](figures/x.png) |
| Schito and Zasso (2014) | OpenFOAM + ? | ![](figures/x.png) | ![](figures/x.png) |
| Churchfield et al. (2012) | OpenFOAM + SOWFA | ![](figures/x.png) | ![](figures/check.png) |

[Sorensen and Shen (2002)]: http://doi.org/10.1115/1.1471361

Note how rare it is to share code. Papers show equations and results, but in order to build upon the work, researchers must re-implement, debug, etc. This is wasteful and inefficient. There are hardly any good reasons for this practice, but plenty of bad ones.

I would have much rather contributed to someone else's code than start my own from scratch. While keeping your work secret ("hoarding" knowledge) may prevent people from using it first, it also prevents people from helping and collaborating with you. 


## NREL's [SOWFA](https://nwtc.nrel.gov/SOWFA)

* Built on top of OpenFOAM
* Atmospheric BL modeling tools
* Additional LES models
* Generator torque control
* Blade pitch control
* Coupling with NREL's FAST
* HAWT farm only (no cross-flow turbines)
* ALM requires custom solvers

Adapt for CFTs?


## [turbinesFoam](https://github.com/turbinesFoam/turbinesFoam)

* New actuator line modeling library for OpenFOAM
* Subclasses OpenFOAM's `fvOptions`
  * Works with standard solvers
* Written in object-oriented style
  * Turbine, actuator line, and actuator line element are all software objects
* Model both cross- and axial-flow turbines, standalone actuator lines


## Distinctive features

* Inflow sampled away from element using cell-point interpolation
* Smearing parameter $\epsilon$ either $c/2$ or $2 \sqrt[3]{V_\mathrm{cell}}$ (whichever larger)
* Three variants of the Leishman–Beddoes dynamic stall model
* Flow curvature corrections for CFTs
* Added mass model from Strickland (1981)
* Lifting-line based end effects model (no rotor parameters needed)—see Anderson (2001):

$$
\alpha(\theta) = \frac{2b}{\pi c(\theta)} \sum_1^N A_n \sin n \theta + \sum_1^N n A_n \frac{\sin n \theta}{\sin \theta}
+ \alpha_{L=0}(\theta)
$$

$$
\Gamma(\theta) = 2b U_\infty \sum_1^N A_n \sin n \theta \hspace{1in} F = \Gamma(\theta)/\Gamma_\max
$$

## OpenFOAM's `fvOptions` framework

Generic mechanism for adding sources at run time:

```c++
// Solve the Momentum equation

tmp<fvVectorMatrix> UEqn
(
    fvm::ddt(U)
  + fvm::div(phi, U)
  + turbulence->divDevReff(U)
 ==
    fvOptions(U)
);
```

Leverage existing solvers---compressible, incompressible, VOF.

Generic turbulence modeling--RANS, LES, DES, etc.

## UNH-RVAT cross-flow turbine validation case

<img width=44% class="float-right" src="figures/rvat-cad-no-hubs.PNG">

Open datasets from tow tank measurements:<br>
[github.com/UNH-CORE/RVAT-baseline](https://github.com/UNH-CORE/RVAT-baseline) <br>
[github.com/UNH-CORE/RVAT-Re-dep](https://github.com/UNH-CORE/RVAT-Re-dep)

3-D standard $k$–$\epsilon$ RANS model

Leishman–Beddoes dynamic stall model modified by Sheng et al. (2008)

Flow curvature correction from Goude (2012)

Added mass model from Strickland (1981)

## UNH-RVAT actuator line simulation

<img width=44% class="float-right" src="figures/rvat-cad-no-hubs.PNG">

3-D standard $k$–$\epsilon$ RANS model

Leishman–Beddoes dynamic stall model modified by Sheng et al. (2008)

Flow curvature correction from Goude (2012)

Added mass model from Strickland (1981)

## Preliminary results: UNH-RVAT performance

<center>
<img width=80% src=figures/UNH-RVAT-turbinesFoam_perf-curves.png>
</center>

## Wake profiles at $x/D = 1$

<center>
<img width=80% src=figures/UNH-RVAT-turbinesFoam_wake-profiles.png>
</center>

## Mean momentum transport

$$
\begin{split}
\frac{\partial U}{\partial x}  =  
\frac{1}{U} \bigg{[}
& - V\frac{\partial U}{\partial y}
- W\frac{\partial U}{\partial z} \\
& -\frac{1}{\rho}\frac{\partial P}{\partial x} \\
& - \frac{\partial}{\partial x} \overline{u'u'}
- \frac{\partial}{\partial y} \overline{u'v'}
- \frac{\partial}{\partial z} \overline{u'w'} \\
& + \nu\left(\frac{\partial^2 U}{\partial x^2}
+ \frac{\partial^2 U}{\partial y^2}
+ \frac{\partial^2 U}{\partial z^2} \right)
\bigg{]}.
\end{split}
$$

## Mean velocity at $x/D=1$ (ALM vs. experiment)

<center>
<img width=65%, src="figures/UNH-RVAT-turbinesFoam_meancontquiv.png">
</center>

<center>
<img width=65%, style="padding-left: 45px", src="figures/br-cfd-meancontquiv_exp.png">
</center>

## UNH-RVAT TKE at $x/D=1$ (ALM vs. experiment)

<center>
<img width=80%, src="figures/UNH-RVAT-turbinesFoam_kcont.png">
</center>

<center>
<img width=80%, src="figures/br-cfd-kcont_exp.png">
</center>

## Near-wake momentum transport

<center>
<img width=90%, src="figures/UNH-RVAT-turbinesFoam_recovery-bar-chart.png">
</center>

## UNH-RVAT cross-flow turbine LES

<center>
<video width=100% controls loop>
  <source src="videos/unh-rvat-alm-les.ogv" type="video/ogg">
Your browser does not support the video tag.
</video>
</center>

<!--
<center>
<iframe width="1000" height="500" src="https://www.youtube.com/embed/THZvV4R1vow?rel=0&autoplay=0" frameborder=0 allowfullscreen></iframe>
</center>
-->

## UNH-RVAT with free surface


<center>
<video width=90% controls loop>
  <source src="videos/unh-rvat-alm-free-surface.ogv" type="video/ogg">
Your browser does not support the video tag.
</video>
</center>

## Sandia/DOE RM2 cross-flow turbine case

<img width=45% class="float-right" src=figures/RM2-rendering.PNG>

Open dataset for low-solidity CFT acquired in UNH tow tank

<img width=50% src=figures/rm2_video_snap.png>

[github.com/UNH-CORE/RM2-tow-tank](https://github.com/UNH-CORE/RM2-tow-tank)


## Preliminary results: RM2 performance

<center>
<img width=80% src=figures/RM2-turbinesFoam_perf-curves.png>
</center>

## NTNU dual in-line HAWTs

Extensive axial-flow turbine validation case by Pierella et al. (2014)

<center>
<img width=70% src=figures/Pierella2014-exp-setup.PNG>
</center>

## NTNU HAWTs performance

Preliminary results with standard $k$–$\epsilon$ RANS model

<center>
<img width=90% src=figures/NTNU-HAWT-turbinesFoam_perf-curves.png>
</center>

## NTNU HAWTs wake profiles

$1D$ behind downstream turbine

<center>
<img width=90% src=figures/NTNU-HAWT-turbinesFoam_wake-profiles.png>
</center>

## NTNU HAWT large eddy simulation

Preliminary results with one equation eddy viscosity model

<center>
<video width=80% controls loop>
  <source src="videos/ntnu-hawt-alm-les.ogv" type="video/ogg">
Your browser does not support the video tag.
</video>
</center>

## Conclusions

* New open-source ALM developed for both AFTs and CFTs in OpenFOAM
  * Compatible with RANS, LES turbulence models and VOF solvers
* Predicts performance reasonably well
* CFT near-wake predictions need work
  * Turbulence generated by dynamic stall
  * Three-dimensionality from vortex shedding

## To-do

Blade BL turbulence "injection" ([calibration via 2-D RANS](https://github.com/petebachant/NACAFoil-OpenFOAM))

<center>
<img width=80% src=figures/turbulence-injection.PNG>
</center>

## To-do

* CFT vorticity generation
  * Blade-vortex interaction?
  * Stall vortex shedding (large negative $C_m$)?

<center>
<img width=80% src=figures/3D_vorticity_SA_964_10-threshold.png>
</center>



## Acknowledgements


<img width=150px src=figures/nsf.png><img width=150px src=figures/us-doe.png><img width=150px src=figures/snl_logo.png>

To obtain or contribute to the code, see https://github.com/turbinesFoam/turbinesFoam

Test cases and more available at http://git.io/v4Bzp

This presentation available at